In [ ]:
import sys
import nanohubremote as nr
import nanohubuidl
import warnings
warnings.filterwarnings("ignore")
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"] 

In [ ]:
tool = nr.Sim2l(auth_data)

In [ ]:
SIMTOOLNAME = "st4pnjunction"
schema = tool.getSchema(SIMTOOLNAME)
params = tool.getToolParameters(SIMTOOLNAME)
#schema

## Importing libraries

In [ ]:
import sys
import json
import nanohub.uidl.teleport as t
from nanohub.uidl.simtool import SimtoolBuilder
from nanohub.uidl.material import MaterialBuilder
from nanohub.uidl.material import MaterialContent, MaterialLabContent
from nanohub.uidl.plotly import PlotlyBuilder
from nanohub.uidl.app import AppBuilder

from nanohub.uidl.nanohub import Auth
TOOLNAME = schema["name"]# "st4pnjunction"
REVISION = 12 #schema["revision"]
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_LOGIN_OPEN = "login_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"

## Creating the project and adding GUI variables 

In [ ]:
Project = t.TeleportProject("PNToyLab")
Component = Project.root
Component.addStateVariable("DrawerIsVisible", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("AppBarStyle", {"type":"string", "defaultValue": "bar_shifted"})
Component.addStateVariable("open_results", {"type":"boolean", "defaultValue": False})
Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("plotly_shift", {"type":"object", "defaultValue": {
  "height": "calc(-80px + 100vh)",
  "width": "calc(-350px + 100vw)",
  "right": "0px",
  "position":"absolute"
}})


In [ ]:
Project.libraries['react'] = 'https://nanohub.org/js_apps/react.production.min'
Project.libraries['react-dom'] = 'https://nanohub.org/js_apps/react-dom.production.min'
Project.libraries['material-ui'] = 'https://nanohub.org/js_apps/material-ui.production.min'
Project.libraries['materiallab-ui'] = 'https://nanohub.org/js_apps/material-ui-lab.production.min'
Project.libraries['plotlycomponent'] = 'https://nanohub.org/js_apps/create-plotly-component'
Project.libraries['plotly'] = 'https://nanohub.org/js_apps/plotly-latest.min'
Project.libraries['math'] = 'https://cdnjs.cloudflare.com/ajax/libs/mathjs/6.6.1/math.min'
Project.libraries['axios'] = 'https://nanohub.org/js_apps/axios.min'
Project.libraries['localforage'] = 'https://nanohub.org/js_apps/localforage.min'
#Project.libraries['number-format'] = 'https://nanohub.org/js_apps/react-number-format'
Project.libraries['prop-types'] = 'https://nanohub.org/js_apps/prop-types.min'

## Creating Error

In [ ]:
ErrorMessage = SimtoolBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
    is_open = False
)

## Creating loader

In [ ]:
Loader = SimtoolBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
    is_open = False
)

## Schema

In [ ]:
#SimtoolBuilder.buildSchema(
#    Project,
#    Component,
#    url = "https://nanohub.org/api/dbexplorer/simtools",
#    toolname = TOOLNAME,
#    revision = REVISION
#)
#Component.addPropVariable(
#    "onLoadSchema", 
#    {
#        "type" : "func", 
#        'defaultValue' : 
#        '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'
#    }
#)    

## Authentication

In [ ]:
from secrets import IDCLIENT, SECRET
auth_data['client_id'] = IDCLIENT
auth_data['client_secret'] = SECRET
# to get client_id and client_secret, create a web application (https://nanohub.org/developer/api/applications/new), use "https://127.0.0.1" as Redirect URL   
Login, CLogin = Auth.Login(
    Project,
    Component,
    client_id = auth_data['client_id'],
    client_secret = auth_data['client_secret'],
    url = "https://nanohub.org/api/developer/oauth/token",   
    open_state = STATE_LOGIN_OPEN
)
onRefreshViews = SimtoolBuilder.refreshViews(
   Project, 
   Component 
)   
Login.content.events["onAuth"] = [
    { "type": "stateChange", "modifies": STATE_LOGIN_OPEN, "newState": False},
    #{ "type": "propCall2", "calls": "buildSchema", "args": ['self'] },
    onRefreshViews[0],
]

In [ ]:
Drawer = MaterialBuilder.Drawer(Project, state="DrawerIsVisible", variant="persistent" )
Drawer = t.TeleportElement(MaterialContent(elementType="Drawer"))
Drawer.content.attrs["variant"] = "persistent"
Drawer.content.attrs["anchor"] = "left"
Drawer.content.attrs["open"] = {
    "type": "dynamic",
    "content": {
      "referenceType": "state",
      "id": "DrawerIsVisible"
    }    
}


In [ ]:
import nanohub.remote as nr
tool = nr.Sim2l(auth_data, url="https://nanohub.org/api/")
params = tool.getToolParameters(TOOLNAME)
SETTINGS = {}
for option, value in params.items():
    if isinstance(value,nr.params.Number):
        SETTINGS[option] = {
            "type": "Number",
            "default_value": value.default, 
            "units" : value.units, 
            "min" : value.min, 
            "max" : value.max, 
            "label" : value.label,
            "description" : value.description,
        }   
    elif isinstance(value,nr.params.Integer):
        SETTINGS[option] = {
            "type": "Integer",
            "default_value": value.default, 
            "units" : value.units, 
            "min" : value.min, 
            "max" : value.max, 
            "label" : value.label,
            "description" : value.description,
        }   
    elif isinstance(value,nr.params.String):
        SETTINGS[option] = {
            "type": "String",
            "default_value": value.default, 
            "label" : value.label,
            "description" : value.description,
        }   
    elif isinstance(value,nr.params.Choice):
        SETTINGS[option] = {
            "type": "Select",
            "default_value": value.default, 
            "options": {k : k for k in value.options}, 
            "units" : value.units, 
            "label" : value.label,
            "description" : value.description,
        }
    elif isinstance(value,nr.params.Boolean):
        SETTINGS[option] = {
            "type": "Boolean",
            "default_value": (value.default == "yes"),
            "description" : value.description,
            "label" : value.label,
        }
        
SETTINGS["cores"]  = {
    "type": "Integer",
    "default_value": 1,
    "units" : "", 
    "min" : 1, 
    "max" : 8,     
    "description" : "Number of cores in in the venue",
    "label" : "Number of cores",
}
SETTINGS["cutoff"] = {
    "type": "Integer",
    "default_value": 15,
    "units" : "mins", 
    "min" : 1, 
    "max" : 200,     
    "description" : "Time cutoff",
    "label" : "Time cutoff",
}
SETTINGS["venue"] = {
    "type": "String",
    "default_value": "headnode@brown_tapis_split",
    "description" : "Venue identifier",
    "label" : "Venue identifier",
}
#SETTINGS     


## Defining Layout

In [ ]:
LAYOUT = {'input': {'type': 'tab',
  'id': '',
  'label': '',
  'layout': 'horizontal',
  'children': [{'type': 'group',
    'label': 'Structure',
    'enable': None,
    'layout': 'horizontal',
    'children': [{'type': 'number',
      'id': 'p_len',
      'label': 'P-type length',
      'enable': None},
     {'type': 'integer',
      'id': 'p_node',
      'label': 'P-type Nodes',
      'enable': None},
     {'type': 'number',
      'id': 'i_len',
      'label': 'Intrinsic Region length',
      'enable': None},
     {'type': 'integer',
      'id': 'i_node',
      'label': 'Intrinsic Nodes',
      'enable': None},
     {'type': 'number',
      'id': 'n_len',
      'label': 'N-type length',
      'enable': None},
     {'type': 'integer',
      'id': 'n_node',
      'label': 'N-type Nodes',
      'enable': None},
     {'type': 'number',
      'id': 'Na',
      'label': 'Acceptor concentration (Na-)',
      'enable': None},
     {'type': 'number',
      'id': 'Nd',
      'label': 'Donor concentration (Nd+)',
      'enable': None}
   ]},           
   {'type': 'group',
    'label': 'Materials',
    'enable': None,
    'layout': 'horizontal',
    'children': [{'type': 'choice',
      'id': 'materialp',
      'label': 'Material',
      'enable': None},
     {'type': 'group',
      'label': 'Minority carrier lifetimes',
      'enable': None,
      'layout': 'horizontal',
      'children': [{'type': 'number',
        'id': 'taun',
        'label': 'For electrons',
        'enable': None},
       {'type': 'number',
        'id': 'taup',
        'label': 'For holes',
        'enable': None}]},
     {'type': 'boolean',
      'id': 'impurity',
      'label': 'Impurity doping in Intrinsic region.',
      'enable': None},
     {'type': 'choice',
      'id': 'impuritydoping',
      'label': 'Type of doping',
      'enable': [{'operand': 'impurity',
        'operator': '==',
        'value': True,
        'condition': ''}]},
     {'type': 'number',
      'id': 'impuritylevel',
      'label': 'Doping level',
      'enable': [{'operand': 'impurity',
        'operator': '==',
        'value': True,
        'condition': ''}]}]},
   {'type': 'group',
    'label': 'Environment',
    'enable': None,
    'layout': 'horizontal',
    'children': [{'type': 'number',
      'id': 'temperature',
      'label': 'Ambient temperature',
      'enable': None},
     {'type': 'number',
      'id': 'vsweep_high',
      'label': 'Applied Voltage',
      'enable': None},
     {'type': 'integer',
      'id': 'vn_step',
      'label': 'Number of points',
      'enable': None}]},
    {'type': 'group',
    'label': 'Advanced',
    'enable': None,
    'layout': 'horizontal',
    'children': [{'type': 'number',
      'id': 'cores',
      'label': 'Number of cores',
      'enable': None},
     {'type': 'number',
      'id': 'cutoff',
      'label': 'Time Cutoff',
      'enable': None},
     {'type': 'x',
      'id': 'venue',
      'label': 'Venue ID',
      'enable': None}]}]}}

## Creating the settings panel

In [ ]:
url_sim = "https://nanohub.org/api/dbexplorer/simtools"
PNTOYSettings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    revision = REVISION,
    layout = LAYOUT['input'],
    outputs = ['Ec', 'Ev', 'Ei', 'Efn', 'Efp', 
               'Total Current Holes', 'Total Current Electrons', 'Total Current',
               'Hole Density', 'Electron Density', 'Charge Density',
               'Net Charge Density',
               'Electrostatic Potential',
               'Electric Field',
               'Recombination Rate',
               'Excess Electron Density', 'Excess Hole Density',
               'CV Characteristic', 'IV Characteristic'
    ],
    runSimulation = "simtool"
)
PNTOYSettings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'}
]
PNTOYSettings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]

loadDefaultSimulation = SimtoolBuilder.loadDefaultSimulation(Project, Component);
PNTOYSettings.content.events["onLoad"]=[
    loadDefaultSimulation
]

PNTOYSettings.content.events["submit"] = [
    { "type": "stateChange", "modifies": "parameters","newState": '$e.target.value'}
]

PNTOYSettings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]
PNTOYSettings.content.events["onSuccess"]=[
  { "type": "stateChange", "modifies": "open_plot","newState":'$self.state.open_plot'},
  { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
  { "type": "stateChange", "modifies": "open_results","newState": True},
  { "type": "stateChange", "modifies": "open_params","newState": False},
  { "type": "stateChange", "modifies": "active_cache" , 
   "newState": "$self.state.active_cache.filter(f => f !== arguments[1]).concat(arguments[1])" },
  onRefreshViews[0],
]

## Creating the output options

In [ ]:
SimtoolBuilder.loadXY(Project, Component)
SimtoolBuilder.loadSequence(Project, Component)

Project.root.addStateVariable("visualization", {"type":"object", "defaultValue": {
    'function':'loadSequence', 
    'dataset' : ['Ec', 'Ev', 'Ei'],
    'layout' : {},
    'parameters' : {'position':'position','function':'function'}
}})

RESULTS = {
    "bands" : {
        'title' : 'Energy Band Diagram',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Ec', 'Ev', 'Ei'],
            'layout' : {'title':'Energy Band Diagram', 'yaxis': { 'title' : 'Energy (eV)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'function'},
            'normalize' : True,
            'start_trace' : 0
        },
            "callbacks" : onRefreshViews
        }
    },
    "cv" : {
        'title' : 'C-V Characteristics',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadXY', 
            'dataset' : ['CV Characteristic'],
            'layout' : {'title':'CV Characteristics', 'yaxis': { 'type' : 'log', 'title' : 'Capacitance(F/cm^2)', 'exponentformat':'e' }, 'xaxis': { 'title' : 'Gate Voltage (V)' }  },
            'parameters' : {'position':'voltage','function':'function'}
        },
            "callbacks" : onRefreshViews
        }
    },
    "iv" : {
        'title' : 'I-V Characteristics',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadXY', 
            'dataset' : ['IV Characteristic'],
            'layout' : {'title':'IV Characteristics', 'yaxis': {'title' : 'Current(A/cm^2)', 'exponentformat':'e' }, 'xaxis': { 'title' : 'Gate Voltage (V)' }  },
            'parameters' : {'position':'voltage','function':'function'}
        },
            "callbacks" : onRefreshViews
        }
    },
    "current" : {
        'title' : 'Electron and Hole Current',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Total Current Holes', 'Total Current Electrons', 'Total Current'],
            'layout' : {'title':'Total, Electron and Hole Current', 'yaxis': { 'type' : 'log', 'title' : 'Current Density (\cm3)' , 'exponentformat':'e'} , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'function'},
            'normalize' : True,
            'start_trace' : 1
        },
            "callbacks" : onRefreshViews
        }
    },
    "density" : {
        'title' : 'Electron and Hole Density',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Hole Density', 'Electron Density', 'Charge Density'],
            'layout' : {'title':'Doping, Electron and Hole Density', 'yaxis': { 'type':'log' ,'title' : 'Charge Density (\cm3)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'function'},
            'normalize' : True,
            'start_trace' : 0
        },
            "callbacks" : onRefreshViews
        }
    },
    "net" : {
        'title' : 'Net Charge Density',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Net Charge Density'],
            'layout' : {'title':'Net Charge Density', 'yaxis': { 'title' : 'Charge Density (C\cm3)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'function'},
            'normalize' : True,
            'start_trace' : 0
        },
            "callbacks" : onRefreshViews
        }
    },
    "potential" : {
        'title' : 'Electrostatic Potential',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Electrostatic Potential'],
            'layout' : {'title':'Electrostatic Potential', 'yaxis': { 'title' : 'Potential (V)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'delta'},
            'normalize' : True,
            'start_trace' : 0
        },
            "callbacks" : onRefreshViews
        }
    },
    "field" : {
        'title' : 'Electric Field',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Electric Field'],
            'layout' : {'title':'Electrostatic Field', 'yaxis': { 'title' : 'Electric Fiueld (V/cm)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'function'},
            'normalize' : True,
            'start_trace' : 0
        },
            "callbacks" : onRefreshViews
        }
    },
    "recombination" : {
        'title' : 'Recombination Rate',
        'action' : { "type": "stateChange", "modifies": "visualization","newState": {
            'function':'loadSequence', 
            'dataset' : ['Recombination Rate'],
            'layout' : {'title':'Recombination Rate', 'yaxis': { 'title' : 'Recombination Rate (/cm^3s)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
            'parameters' : {'position':'position','function':'function'},
            'normalize' : True,
            'start_trace' : 0
        },
            "callbacks" : onRefreshViews
        }
    },
    "carrier" : {
        'title' : 'Excess Carrier Density',
        'action' : { 
            "type": "stateChange", 
            "modifies": "visualization",
            "newState": {
                'function':'loadSequence', 
                'dataset' : ['Excess Electron Density', 'Excess Hole Density'],
                'layout' : {'title':'Excess Carrier Density', 'yaxis': { 'type' : 'log', 'title' : 'Carrier Density (/cm^3)', 'exponentformat':'e' } , 'xaxis': { 'title' : 'Position(um)' } },
                'parameters' : {'position':'position','function':'function'},
                'normalize' : True,
                'start_trace' : 1
            },
            "callbacks" : onRefreshViews
        }
    },
}



In [ ]:
PNTOYResults = AppBuilder.Results( 
    Component,
    results = RESULTS,
    onClick = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True }],
    onLoad = [
        { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False }
    ],
)

## Creating Colorschema and adding nanoHUB logo

In [ ]:
ThemeProvider = MaterialBuilder.ThemeProvider( Component, MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',
))
AppBar = MaterialBuilder.AppBar(
    state="DrawerIsVisible", 
    styles=("plotly_shift", [
        {
          "height": "calc(-80px + 100vh)",
          "width": "calc(-350px + 100vw)",
          "right": "0px",
          "position":"absolute"
        },
        {
          "height": "calc(-80px + 100vh)",
          "width": "calc(0px + 100vw)",
          "right": "0px",
        } 
    ]), 
    title="PNToy lab",
    callbacks = onRefreshViews
)
logo = t.TeleportElement(t.TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)


## Creating Expansion panels

In [ ]:
#Project.globals.assets.append({"type": "style", "content": ".bar_shifted {margin-left:350px; width: calc(100% - 350px);}"})
Project.globals.assets.append({"type": "style", "content": ".bar_shifted {margin-left:0px;}"})
Project.globals.assets.append({ "type": "style", "content": ".bar_normal {margin-left:0px}"})
ExpansionPanel = MaterialBuilder.ExpansionPanel(
    title="Results", 
    disabled=False,
    content=[PNTOYResults],
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_results"
      }    
    },     
)
ExpansionPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_results","newState": "$toggle"}]
Drawer.addContent(ExpansionPanel)

ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    disabled=False, 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[PNTOYSettings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]
Drawer.addContent(ParametersPanel)


## Creating reset button

In [ ]:
Gridt = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridt.content.attrs["color"] = "secondary"
Gridt.content.attrs["container"] = True
Gridt.content.attrs["direction"] = "column"
resetSettings = {}
for k,v in SETTINGS.items():
    resetSettings[k] = v["default_value"]
Buttontt= MaterialBuilder.Button(
    title = "Reset Setting", 
    variant = "text", 
    style = {'backgroundColor':'#999999', 'borderRadius' : '0px', 'minHeight':'40px', 'color':'rgba(255, 255, 255, 0.87)'}, 
    onClickButton=[{ "type": "stateChange", "modifies": k,"newState": v["default_value"]}
        for k,v in SETTINGS.items()          
    ]
  )
Gridt.addContent(Buttontt)
Project.components['AppSettingsComponent'].node.addContent(Gridt)

## Creationg ploting component

In [ ]:
BasePlot = PlotlyBuilder.BasePlot( Project, Component, style_state="AppBarStyle")
BasePlot.content.style = "$self.state.plotly_shift"


In [ ]:
IconText = t.TeleportStatic()
IconText.content = "$local.icon"

Icon = t.TeleportElement(MaterialContent(elementType="Icon"))
Icon.content.style = "$local.style"
Icon.addContent(IconText)   

ToggleButton = t.TeleportElement(MaterialLabContent(elementType="ToggleButton"))
ToggleButton.content.attrs['value'] = "$local.id"
ToggleButton.addContent(Icon)
ToggleButton.content.events['mouseenter'] = [
    { "type": "stateChange", "modifies": "anchorEl" ,"newState": "$e.currentTarget" },
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": True },
    { "type": "stateChange", "modifies": "lastSelected" ,"newState": "$[local.id]" },
    { "type": "stateChange", "modifies": "lastColor" ,"newState": "$local.style" },
    { "type": "stateChange", "modifies": "lastDescription" ,"newState": "$local.description" },
]
ToggleButton.content.events['click'] = [
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": False, "callbacks" : onRefreshViews}
]

RepeatIcons = t.TeleportRepeat(ToggleButton)
RepeatIcons.iteratorName = "local"
RepeatIcons.dataSource = {
    "type": "dynamic",
    "content": {
      "referenceType": "state",
      "id": "cached_results"
    }    
}

ToggleButtonGroup = t.TeleportElement(MaterialLabContent(elementType="ToggleButtonGroup"))
ToggleButtonGroup.addContent(RepeatIcons)
#ToggleButtonGroup.content.attrs['size'] = "large"
ToggleButtonGroup.content.attrs['value'] = {
  "type": "dynamic",
  "content": {
    "referenceType": "state",
    "id": 'active_cache'
  }
}
ToggleButtonGroup.content.events["change"] = [
    { "type": "stateChange", "modifies": "active_cache" ,"newState": "$arguments[1]" },
    { "type": "stateChange", "modifies": "anchorEl" ,"newState": "$e.currentTarget" },
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": True },
]

Paper = t.TeleportElement(MaterialContent(elementType="Paper"))

Project.root.addStateVariable("cached_results", {"type":"array", "defaultValue": []})
Project.root.addStateVariable("active_cache", {"type":"array", "defaultValue": []})
Project.root.addStateVariable("anchorEl", {"type":"object", "defaultValue": {}})
Project.root.addStateVariable("anchorOpen", {"type":"boolean", "defaultValue": False})
Project.root.addStateVariable("lastSelected", {"type":"array", "defaultValue": []})
Project.root.addStateVariable("lastColor", {"type":"object", "defaultValue": {'color':'#f00'}})
Project.root.addStateVariable("lastDescription", {"type":"string", "defaultValue": ""})


Popper = t.TeleportElement(MaterialContent(elementType="Popper"))
Popper.content.attrs['anchorEl'] = {
  "type": "dynamic",
  "content": {
    "referenceType": "state",
    "id": 'anchorEl'
  }
}
Popper.content.attrs['open'] = {
  "type": "dynamic",
  "content": {
    "referenceType": "state",
    "id": 'anchorOpen'
  }
}

Icon1 = t.TeleportElement(MaterialContent(elementType="Icon"))
Icon1.addContent(t.TeleportStatic(content = "delete"))   
Icon1.content.attrs['style'] = "$self.state.lastColor"


onRemoveCache = SimtoolBuilder.removeCacheRecord(
   Project, 
   Component 
)  


Fab1 = t.TeleportElement(MaterialContent(elementType="IconButton"))
Fab1.content.attrs['size'] = "small"
Fab1.addContent(Icon1)
Fab1.content.events['click'] = [
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": False },
    onRemoveCache[0],
]

Icon2 = t.TeleportElement(MaterialContent(elementType="Icon"))
Icon2.addContent(t.TeleportStatic(content = "visibility"))   
Icon2.content.attrs['style'] = "$self.state.lastColor"

Fab2 = t.TeleportElement(MaterialContent(elementType="IconButton"))
Fab2.content.attrs['size'] = "small"
Fab2.addContent(Icon2)
Fab2.content.events['click'] = [
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": False },
    { 
        "type": "stateChange", 
        "modifies": "active_cache",
        "newState": "$self.state.active_cache.filter(f => f !== self.state.lastSelected[0]).concat(self.state.lastSelected)",
        "callbacks" : onRefreshViews
    }
]

Icon3 = t.TeleportElement(MaterialContent(elementType="Icon"))
Icon3.addContent(t.TeleportStatic(content = "visibility_off"))   
Icon3.content.attrs['style'] = "$self.state.lastColor"

Fab3 = t.TeleportElement(MaterialContent(elementType="IconButton"))
Fab3.content.attrs['size'] = "small"
Fab3.addContent(Icon3)
Fab3.content.events['click'] = [
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": False },
    { 
        "type": "stateChange", 
        "modifies": "active_cache",
        "newState": "$self.state.active_cache.filter(f => f !== self.state.lastSelected[0])",
        "callbacks" : onRefreshViews
    },
]

Gridcontainer = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridcontainer.content.attrs['container'] = True

Griditem1 = t.TeleportElement(MaterialContent(elementType="Grid"))
Griditem1.content.attrs['item'] = True
Griditem1.addContent(Fab1)

Griditem2 = t.TeleportElement(MaterialContent(elementType="Grid"))
Griditem2.content.attrs['item'] = True
Griditem2.addContent(Fab2)

Griditem3 = t.TeleportElement(MaterialContent(elementType="Grid"))
Griditem3.content.attrs['item'] = True
Griditem3.addContent(Fab3)

Gridcontainer.addContent(Griditem1)
Gridcontainer.addContent(Griditem2)
Gridcontainer.addContent(Griditem3)

Typography = t.TeleportElement(MaterialContent(elementType="Typography"))
Typography.content.attrs["variant"] = "inherit"
TypographyText = t.TeleportDynamic(content={"referenceType": "state","id": "lastDescription"})
Typography.addContent(TypographyText)
Typography.content.style = "${'whiteSpace':'pre-line'}"

CardContent = t.TeleportElement(MaterialContent(elementType="CardContent"))
CardContent.addContent(Typography)


CardActions = t.TeleportElement(MaterialContent(elementType="CardActions"))
CardActions.addContent(Gridcontainer)


Card = t.TeleportElement(MaterialContent(elementType="Card"))
Card.addContent(CardActions)
Card.addContent(CardContent)

Popper.addContent(Card)

Card.content.events["mouseleave"] = [
    { "type": "stateChange", "modifies": "anchorOpen" ,"newState": False },
]

#Project.root.addNode(Popper)
LAppBar = t.TeleportElement(MaterialContent(elementType="AppBar"))
LAppBar.content.attrs["position"] = "fixed"
LAppBar.content.attrs["color"] = "transparent"
LAppBar.content.style = {'top': 'auto', 'bottom': '0','width': 'inherit'}

LAppBar.addContent(ToggleButtonGroup)
Paper.addContent(LAppBar)

## Assembling the app

In [ ]:
Gridh = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridh.content.attrs["container"] = True
Gridh.content.attrs["direction"] = "row" 
Gridh.addContent(Drawer)
Gridh.addContent(BasePlot)  

Gridv = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(Paper)  
Gridv.addContent(Gridh)


ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(ErrorMessage)
ThemeProvider.addContent(Login)
ThemeProvider.addContent(Popper)  

Component.addNode(ThemeProvider)
#Project.buildReact( TOOLNAME+"PROD"+".html");

## Displaying the app in jupyter

In [ ]:
#Project.buildReact( TOOLNAME+"PROD"+".html");
from IPython.core.display import display
from IPython.display import IFrame
display(IFrame(src=TOOLNAME+"PROD"+".html", width="100%", height="800"))